In [1]:
# !pip install transformers
!pip install datasets
! pip install -U accelerate
! pip install -U transformers
!pip install evaluate
!pip install sacrebleu
!pip install datasets transformers torch evaluate nltk rouge_score

In [2]:
from datasets import load_dataset

data_files = {
    "train": r"/content/drive/MyDrive/University/PML/A1/Data/training.csv",
    "test": r"/content/drive/MyDrive/University/PML/A1/Data/testing.csv"
}
toxic_dataset = load_dataset("csv", data_files=data_files)
toxic_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 462221
    })
    test: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 115556
    })
})

In [3]:
toxic_dataset["train"][0]

{'Unnamed: 0': 46800,
 'toxic': 'bunch of goddamn cops!',
 'neutral': 'Couple of bloody coppers!',
 'toxicity score': 0.9996676445007324,
 'toxicity of neutral score': 0.0293669588863849}

In [4]:
train_vald_dataset = toxic_dataset["train"].train_test_split(train_size=0.01, test_size=0.01, seed=20)

train_vald_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 4622
    })
    test: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 4623
    })
})

In [5]:
train_vald_dataset["validation"] = train_vald_dataset.pop("test")

In [6]:
train_vald_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 4622
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'toxic', 'neutral', 'toxicity score', 'toxicity of neutral score'],
        num_rows: 4623
    })
})

In [7]:
train_vald_dataset["train"][1]

{'Unnamed: 0': 251439,
 'toxic': 'I wish people would just shut up,',
 'neutral': "he'd want people to be quiet",
 'toxicity score': 0.7318433523178101,
 'toxicity of neutral score': 0.0001017092363326}

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side='left')
input_sentence = train_vald_dataset["train"][1]["toxic"]
target = train_vald_dataset["train"][1]["neutral"]
output = tokenizer(input_sentence, text_target=target)

In [9]:
tokenizer.convert_ids_to_tokens(output["input_ids"])

['I', 'Ġwish', 'Ġpeople', 'Ġwould', 'Ġjust', 'Ġshut', 'Ġup', ',']

In [10]:
max_len = 128

def preprocessing_function(examples):
    inputs = examples["toxic"]
    targets = examples["neutral"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=max_len)
    # For GPT-2, the labels are the input_ids we are trying to predict
    model_inputs["labels"] = tokenizer(targets, padding="max_length", truncation=True, max_length=max_len)["input_ids"]
    return model_inputs


In [11]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = train_vald_dataset.map(
    preprocessing_function,
    batched=True,
    remove_columns=train_vald_dataset["train"].column_names
)

Map:   0%|          | 0/4623 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets["train"][1]

{'input_ids': [50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  5025

In [13]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')

In [14]:
tokenizer.pad_token

'<|endoftext|>'

In [15]:

model.config.pad_token_id = tokenizer.pad_token_id

In [16]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [17]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(10, 13)])
batch.keys()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels'])

In [18]:
batch["labels"]

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  1870,   314,
          1101,  8066,  1309,   345,   734,  1643,  2052,   287,   319,   257,
          1310,  3200,   438,   618,   345,  4341,  1105,   812,   319,   257,
          1241,    12,    19,  3770, 12699,    11,   345,  1716,  2407, 16245,
           286,  1310, 25776,     6,  4813,   588, 2

In [19]:
import evaluate

metric = evaluate.load("sacrebleu")

In [20]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [21]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.35.0', '0.24.1')

In [30]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"gpt2-detoxification",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [31]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss
500,3.430300


In [ ]:
trainer.evaluate(max_length=max_len)

In [ ]:
model_path = r"/content/drive/MyDrive/University/PML/A1/gpt-2"
trainer.save_model(model_path)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the trained model and tokenizer
model_name_or_path = r"/content/drive/MyDrive/University/PML/A1/gpt-2"  # replace with your model's path
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Prepare the prompt
prompt = "I feel like shit today"

# Encode the prompt and move tensors to the same device as the model
inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=128)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate output sequence
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=128, num_return_sequences=1)

# Decode and print the output
detoxified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Detoxified text: {detoxified_text}")
